# Notes

Here I tried to make the code compatible to Tensorflow 2(like with aplha and beta version). You can try to run it in the original tf-1 version of the code

I used a subset of imagenet  and trained the model but the results didn't color at all. In the beta version I got better results.
Maybe with more training it can do better than beta version.


In [32]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers import BatchNormalization

from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers import RepeatVector, Permute

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave,imshow
import numpy as np
import os
import random

In [18]:
test_dir = 'test_image_net/'
train_dir = 'train_image_net/'

In [19]:
# Get images
X = []
for filename in os.listdir('train_image_net/'):
    if filename =='.DS_Store':
        continue
    X.append(img_to_array(load_img('train_image_net/'+filename)))
X = np.array(X, dtype=float)
Xtrain = 1.0/255*X


#Load weights
inception = InceptionResNetV2(weights='imagenet', include_top=True)

In [21]:
embed_input = Input(shape=(1000,))

#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)

#Fusion
fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
fusion_output = concatenate([encoder_output, fusion_output], axis=3) 
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output) 

#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)

In [29]:
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = [resize(i, (299, 299, 3), mode='constant') for i in grayscaled_rgb]
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    embed = inception.predict(grayscaled_rgb_resized)
    return embed

# Image transformer
datagen = ImageDataGenerator(shear_range=0.2, zoom_range=0.2, rotation_range=20, horizontal_flip=True)

# Generate training data
batch_size = 10

def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        grayscaled_rgb = gray2rgb(rgb2gray(batch))
        embed = create_inception_embedding(grayscaled_rgb)
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:, :, :, 0]
        X_batch = X_batch.reshape(X_batch.shape + (1,))
        Y_batch = lab_batch[:, :, :, 1:] / 128
        yield ([X_batch, embed], Y_batch)



1/1 [==============================] - 3s 3s/step - loss: 0.6241


In [ ]:
model.compile(optimizer='rmsprop', loss='mse')
model.fit(image_a_b_gen(batch_size), epochs=1, steps_per_epoch=1)

In [23]:
# def create_inception_embedding(grayscaled_rgb):
#     grayscaled_rgb_resized = []
#     for i in grayscaled_rgb:
#         i = resize(i, (299, 299, 3), mode='constant')
#         grayscaled_rgb_resized.append(i)
#     grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
#     grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
#     with inception.graph.as_default():
#         embed = inception.predict(grayscaled_rgb_resized)
#     return embed

# # Image transformer
# datagen = ImageDataGenerator(
#         shear_range=0.2,
#         zoom_range=0.2,
#         rotation_range=20,
#         horizontal_flip=True)

# #Generate training data
# batch_size = 10

# def image_a_b_gen(batch_size):
#     for batch in datagen.flow(Xtrain, batch_size=batch_size):
#         grayscaled_rgb = gray2rgb(rgb2gray(batch))
#         embed = create_inception_embedding(grayscaled_rgb)
#         lab_batch = rgb2lab(batch)
#         X_batch = lab_batch[:,:,:,0]
#         X_batch = X_batch.reshape(X_batch.shape+(1,))
#         Y_batch = lab_batch[:,:,:,1:] / 128
#         yield ([X_batch, create_inception_embedding(grayscaled_rgb)], Y_batch)


# #Train model      
# model.compile(optimizer='rmsprop', loss='mse')
# model.fit(image_a_b_gen(batch_size), epochs=1, steps_per_epoch=1)

In [30]:
color_me = []
for filename in os.listdir(test_dir):
    if filename == '.DS_Store':
        continue
    color_me.append(img_to_array(load_img(test_dir+filename)))
color_me = np.array(color_me, dtype=float)
gray_me = gray2rgb(rgb2gray(1.0/255*color_me))
color_me_embed = create_inception_embedding(gray_me)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))


# Test model
output = model.predict([color_me, color_me_embed])
output = output * 128

# Output colorizations
# for i in range(len(output)):
#     print(i)
#     print(color_me.shape)
#     cur = np.zeros((256, 256, 3))
#     cur[:,:,0] = color_me[i][:,:,0]
#     cur[:,:,1:] = output[i]
#     imsave("result/img_"+str(i)+".png", lab2rgb(cur))

from skimage import io, color

# this code fixes the above block of code but produces a warning
# for i in range(len(output)):
#     cur = np.zeros((256, 256, 3))
#     cur[:,:,0] = color_me[i][:,:,0]
#     cur[:,:,1:] = output[i]

#     # Ensure the values are in the range 0-255
#     cur = np.clip(cur, 0, 255)

#     # Convert from LAB to RGB
#     cur = color.lab2rgb(cur)

#     # Convert to uint8 format
#     cur = (cur * 255).astype(np.uint8)

#     # Save the image
#     io.imsave("result/img_" + str(i) + ".png", cur)


# this fixes the warning as well
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]  # L channel
    cur[:,:,1:] = output[i]  # a and b channels

    # Clip the a and b channels to a range (-128, 127) that's more likely to convert cleanly
    cur[:,:,1:] = np.clip(cur[:,:,1:], -128, 127)

    # Convert from LAB to RGB, handling clipping internally
    cur = color.lab2rgb(cur)

    # Convert to uint8 format
    cur = (cur * 255).astype(np.uint8)

    # Save the image
    io.imsave("result/img_" + str(i) + ".png", cur)

4/4 [==============================] - 1s 136ms/step
